In [ ]:
import random
import numpy as np
import pandas as pd
from datetime import timedelta
from dask import dataframe as dd
pd.options.mode.chained_assignment = None

train = False

In [ ]:
transactions = dd.read_csv('data/transactions_train.csv', dtype={'article_id': str, 'customer_id': str})
if train:
    transactions = transactions[(transactions.t_dat >= '2020-03-17') & (transactions.t_dat <= '2020-09-15')]
    path = 'data/ensemble_train/'
else:
    path = 'data/ensemble/'
    transactions = transactions[transactions.t_dat >= '2020-03-24']
transactions.t_dat = dd.to_datetime(transactions.t_dat) - timedelta(2)
transactions["week"] = transactions.t_dat.dt.isocalendar().week
transactions = transactions.compute()
transactions["rebuy_count"] = transactions.groupby(["customer_id", "article_id"]).cumcount().astype(int)
transactions["rebuy_count"] = transactions.rebuy_count.apply(lambda x: x -1 if x > 0 else 0)

In [ ]:
most_solds = transactions[transactions.week > transactions.week.max()-3].groupby(["article_id"]).agg({"article_id":"count"})\
                                   .rename(columns={"article_id":"_count"}).reset_index()\
                                   .sort_values('_count', ascending=False)
most_solds = most_solds.head(10000)
transactions = transactions[transactions.article_id.isin(most_solds.article_id)]
last_week_articles = transactions[transactions.week == transactions.week.max()].article_id.unique()
articles = pd.read_csv("data/articles.csv", dtype={'article_id': str})
articles.drop_duplicates(subset=['article_id'], inplace=True)
articles = articles[(articles.article_id.isin(most_solds.article_id)) & (articles.article_id.isin(last_week_articles))]
del most_solds, last_week_articles

In [ ]:
active_customers = transactions.customer_id.unique()
customers = pd.read_csv("data/customers.csv", dtype={'customer_id': str})
customers.drop_duplicates(subset=['customer_id'], inplace=True)
customers = customers[customers.customer_id.isin(active_customers)]
del active_customers

In [ ]:
def gender_classification(age):
    if age < 19:
        return 0
    elif age < 29:
        return 1
    elif age < 49:
        return 2
    elif age < 59:
        return 3
    elif age < 69:
        return 4
    else:
        return 5

customers["age"] = customers.age.fillna(np.mean(customers.age))
customers["age_bin"] = customers.age.map(gender_classification)

In [ ]:
def gender_classification(section_name):
    if "womens" in section_name or "girl" in section_name or "ladies" in section_name:
        return "woman"
    elif "men" in section_name or "boy" in section_name or "boys" in section_name:
        return "man"
    else:
        return "other"

articles.section_name = articles.section_name.map(lambda x: x.lower())
articles["gender_group"] = articles.section_name.apply(gender_classification)

In [ ]:
transactions = transactions.merge(articles[["article_id", "gender_group"]], on="article_id", how="inner")
transactions = transactions.merge(customers[["customer_id","age_bin"]], on="customer_id", how="inner")
customer_hist = transactions.groupby(by="customer_id").agg({"article_id": lambda x: list(x.values), "week": lambda x: list(x.values), "gender_group": lambda x : x.mode().iloc[0], "rebuy_count": "mean"}).reset_index()
customers = customers.merge(customer_hist, on="customer_id", how="left")
customers.article_id = customers.article_id.fillna("").apply(list)
customers.gender_group = customers.gender_group.fillna("other")
transactions.price = transactions.price.fillna(transactions.price.mean())
prod_price = transactions.groupby("customer_id").agg({"price":"mean"}).rename(columns={"customer_id":"price_"}).reset_index()
prod_price.columns = list(map(''.join, prod_price.columns.values))
customers = customers.merge(prod_price, on="customer_id", how="inner")
del customer_hist

In [ ]:
customers["history"] = customers.apply(lambda x: sorted(zip(x.week, x.article_id)), axis=1)
customers.FN = customers.FN.fillna(0)
customers.Active = customers.Active.fillna(0)
customers.fashion_news_frequency = customers.fashion_news_frequency.fillna("not_regular")
customers.club_member_status = customers.club_member_status.fillna("no-info")
customers.fashion_news_frequency = customers.fashion_news_frequency.apply(lambda x: "not_regular" if x == "NONE" or x == "None" else x)
customers["numberOfArticles"] = customers.apply(lambda x: len(x.history), axis=1)
customers = customers.drop(columns=["postal_code", "week", "article_id"])

In [ ]:
def map_fashion_news(name):
    return 1 if name == 'not_regular' else 0
def map_club_member(name):
    return 1 if name == 'ACTIVE' else 0

customers.fashion_news_frequency = customers.fashion_news_frequency.map(map_fashion_news)
customers.club_member_status = customers.club_member_status.map(map_club_member)

In [ ]:
def article2doc(x):
    def clean_doc(text):
        unwanted_chars = ['1','2','3','4','5','6','7','8','9','(',')','[',']']
        for chr in unwanted_chars:
            text = text.replace(chr, '')
        return text

    doc =  '. '.join([x.prod_name, x.product_type_name, x.product_group_name, x.graphical_appearance_name, x.colour_group_name,\
                      x.perceived_colour_value_name, x.perceived_colour_master_name, x.department_name, x.index_name, x.index_group_name,\
                      x.section_name, x.garment_group_name, str(x.detail_desc)])[:-1]
    return(clean_doc(doc.lower()))

articles["doc"] = articles.apply(article2doc, axis=1)
article_info = transactions.groupby(["article_id"]).agg({"price":"mean", "rebuy_count":"mean", "age_bin": lambda x : x.mode().iloc[0]}).reset_index()
articles = articles.merge(article_info, on="article_id", how="inner")
most_solds = transactions[transactions.week > transactions.week.max()-3].groupby(["article_id"]).agg({"customer_id":"count"})\
                                      .rename(columns={"customer_id":"prod_sold_count"}).reset_index()
articles = articles.merge(most_solds, on="article_id", how="inner")
articles = articles[["article_id","doc","gender_group", "price", "rebuy_count", "age_bin", "prod_sold_count"]]
weekly_sales = transactions.groupby(["article_id", "week"]).agg({"customer_id":"count"}).reset_index()
last_week_sales = weekly_sales[weekly_sales.week == weekly_sales.week.max()]
weekly_sales = weekly_sales.merge(last_week_sales[["article_id","customer_id"]], on=["article_id"], how="inner")
weekly_sales["quotient"] = weekly_sales.customer_id_y / weekly_sales.customer_id_x
articles = articles.merge(weekly_sales[["article_id","quotient"]], on="article_id", how="inner")

del article_info, most_solds, weekly_sales, last_week_sales

In [ ]:
customers["doc"] = customers.history.apply(lambda x: list(set([i[1] for i in x])))
temp_dict = {}
for i,row in articles.iterrows():
    temp_dict[row.article_id] = row.doc

customers["doc"] = customers.doc.apply(lambda x:  ". ".join([temp_dict[i] for i in x]))
del temp_dict

In [ ]:
customers.head()

In [ ]:
articles.head()

## Content Based Corpus Data Creating

In [ ]:
doc_df1 = articles[["article_id","doc"]].rename(columns={"article_id":"doc_id"}).copy()
articles.drop(columns=["doc"], inplace=True)
doc_df2 = customers[["customer_id","doc"]].rename(columns={"customer_id":"doc_id"}).copy()
customers.drop(columns=["doc"], inplace=True)
doc_df1["type"] = "product"
doc_df2["type"] = "customer"
doc_df = doc_df1.append(doc_df2)
doc_df.to_csv(path+"corpus.csv", index=False)

del doc_df, doc_df1, doc_df2

## Collabrative Data Creating

last_week = 38

In [ ]:
transactions[["customer_id","article_id","week"]].to_pickle("data/transactions.pkl")
last_week = transactions.week.max()

In [ ]:
customers.FN = customers.FN.astype('category').cat.codes
customers.Active = customers.Active.astype('category').cat.codes
customers.club_member_status = customers.club_member_status.astype('category').cat.codes
customers.fashion_news_frequency = customers.fashion_news_frequency.astype('category').cat.codes
customers.age_bin = customers.age_bin.astype('category').cat.codes
customers.gender_group = customers.gender_group.astype('category').cat.codes
customers.age = (customers.age - customers.age.min()) / (customers.age.max() - customers.age.min())
customers.numberOfArticles = (customers.numberOfArticles - customers.numberOfArticles.min()) / (customers.numberOfArticles.max() - customers.numberOfArticles.min())
if train != True:
    customers.drop_duplicates(subset=["customer_id"], inplace=True)
    customers.to_pickle("data/customers.pkl")
    print("Customers saved for prediction...")

In [ ]:
articles.gender_group = articles.gender_group.astype('category').cat.codes
articles.age_bin = articles.age_bin.astype('category').cat.codes
articles.price = (articles.price - articles.price.min()) / (articles.price.max() - articles.price.min())
articles.rebuy_count = (articles.rebuy_count - articles.rebuy_count.min()) / (articles.rebuy_count.max() - articles.rebuy_count.min())
articles.prod_sold_count = (articles.prod_sold_count - articles.prod_sold_count.min()) / (articles.prod_sold_count.max() - articles.prod_sold_count.min())
articles.quotient = (articles.quotient - articles.quotient.min()) / (articles.quotient.max() - articles.quotient.min())
if train != True:
    articles.drop_duplicates(subset=["article_id"], inplace=True)
    articles.to_pickle("data/articles.pkl")
    print("Articles saved for prediction...")

## Ensemble Data Creating

In [ ]:
data = dd.read_csv('data/transactions_train.csv', dtype={'article_id': str, 'customer_id': str})[["customer_id", "article_id", "t_dat"]]
data = data[data.t_dat >= '2020-09-15'].compute()
data["label"] = 1.0
data = data.drop_duplicates(subset=["customer_id", "article_id"])
data.drop(columns=["t_dat"], inplace=True)
article_id_list = list(articles.article_id.values)
data = data[data.article_id.isin(article_id_list)]

In [ ]:
group_info = data.groupby(["customer_id"]).agg({"article_id": lambda x: list(set(x))}).reset_index()
group_info["prod_num"] = group_info.article_id.apply(lambda x: len(x))
customer_id_list = data.customer_id.to_list()
prod_id_list = data.article_id.to_list()
label_list = data.label.to_list()
del data

In [ ]:
for i, row in group_info.iterrows():
    temp_articles = random.sample(article_id_list, 300)
    step = 0
    for id in temp_articles:
        if id not in row.article_id:
            customer_id_list.append(row.customer_id)
            label_list.append(0.0)
            prod_id_list.append(id)
            step += 1
        if step >= 250:
            break
    print('\r' + f'{i}: %{round(100*i/group_info.shape[0], 2)}', end='')

data = pd.DataFrame({"customer_id": customer_id_list, "article_id": prod_id_list, "label": label_list, "week": last_week+1})

In [ ]:
data = dd.from_pandas(data, npartitions=8)
data = data.merge(articles.rename(columns={"age_bin":"prod_age_bin", "gender_group":"prod_gender_group", "rebuy_count":"prod_rebuy_count","price":"prod_avg_price"}), on="article_id", how="inner")
data = data.merge(customers.rename(columns={"age_bin":"customer_age_bin", "gender_group":"customer_gender_group", "rebuy_count":"customer_rebuy_count","price":"customer_avg_price"}), on="customer_id", how="inner")
data["same_prod_rebuy_count"] = data.apply(lambda x: [i[1] for i in x.history].count(x.article_id), axis=1, meta=("same_prod_rebuy_count","int"))
del articles, customers

In [ ]:
def get_last_purchased_time(x):
    week_passed = 7*4
    if x.same_prod_rebuy_count != 0:
        for i in x.history:
            if i[1] == x.article_id:
                week_passed = i[0]
        return x.week - week_passed
    else:
        return week_passed

def get_avg_purchase_time(x):
    if x.same_prod_rebuy_count != 0:
        week = [x.week]
        for i in x.history:
            if i[1] == x.article_id:
                week.append(i[0])
        return (max(week) - min(week)) / (len(week) - 1)
    return 0

In [ ]:
data["time_passed_last_purchase"] = data.apply(lambda x: get_last_purchased_time(x), axis=1,  meta=("time_passed_last_purchase","int"))
data["avg_purchase_time"] = data.apply(lambda x: get_avg_purchase_time(x), axis=1,  meta=("time_passed_last_purchase","float"))
data.drop(columns=["history", "week"])
#data.same_prod_rebuy_count = (data.same_prod_rebuy_count - data.same_prod_rebuy_count.min()) / (data.same_prod_rebuy_count.max() - data.same_prod_rebuy_count.min())

In [ ]:
data.to_csv(path + "ensemble_data.csv")

## Prediction Data Creating

In [ ]:
import random
import numpy as np
import pandas as pd
import cudf as gd
from dask import dataframe as dd

customers = pd.read_pickle("data/ensemble/customers.pkl")
customers.drop_duplicates(subset=["customer_id"], inplace=True)
articles = pd.read_pickle("data/ensemble/articles.pkl")
articles.drop_duplicates(subset=["article_id"], inplace=True)
article_ids = articles.article_id.values.tolist()
customer_ids = customers.customer_id.values

batch_size = 1024
article_ids = article_ids * batch_size
for batch_i in range(batch_size, len(customer_ids), batch_size):
    customer_ids_batch = customer_ids[batch_i-batch_size:batch_i]
    customer_ids_batch = np.repeat(customer_ids_batch, len(article_ids)/batch_size)


    
    df = pd.DataFrame({"customer_id": customer_ids_batch, "article_id": article_ids, "week": last_week+1})
    df = df.merge(articles.rename(columns={"age_bin":"prod_age_bin", "gender_group":"prod_gender_group", "rebuy_count":"prod_rebuy_count","price":"prod_avg_price"}), on="article_id", how="inner")
    df = df.merge(customers.rename(columns={"age_bin":"customer_age_bin", "gender_group":"customer_gender_group", "rebuy_count":"customer_rebuy_count","price":"customer_avg_price"}), on="customer_id", how="inner")
    df["same_prod_rebuy_count"] = df.apply(lambda x: [i[1] for i in x.history].count(x.article_id), axis=1)
    df["time_passed_last_purchase"] = df.apply(lambda x: get_last_purchased_time(x), axis=1)
    df["avg_purchase_time"] = df.apply(lambda x: get_avg_purchase_time(x), axis=1)
    df.drop(columns=["history", "week"])
    break

In [ ]:
df.head()